# 离群点分析与异常检测
仓库地址为：https://github.com/Wchoward/DataMiningClass/tree/master/assignment4

In [1]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [ ]:
def get_dir_filelist(dir):
    filelst = [os.path.join(dir, file) for file in os.listdir(dir)]
    return filelst

## 模型及代码
### 数据获取

获取数据文件列表并排序，并创建roc_df, prn_df, time_df分别用来对roc_auc_score, precision_n_scores和耗费时间 进行存储。

In [ ]:
file_list = get_dir_filelist('../../data/abalone/benchmarks')
file_list.sort()
# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM', 'PCA']
roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

对文件列表中的每个文件进行离群点检测，采用[Python Outlier Detection (PyOD)](https://github.com/yzhao062/pyod)工具包进行离群点检测分析，使用的模型包含：  
- Angle-based Outlier Detector (ABOD)
- Cluster-based Local Outlier Factor
- Feature Bagging
- Histogram-base Outlier Detection (HBOS)
- Isolation Forest
- K Nearest Neighbors (KNN)
- Local Outlier Factor (LOF)
- Minimum Covariance Determinant (MCD)
- One-class SVM (OCSVM)
- Principal Component Analysis (PCA)


In [ ]:
for file in file_list:
    print('----------')
    print('processing file '+ file[-8:-4])
    print('----------')
    df = pd.read_csv(file)
    x = df.drop(['ground.truth','point.id','motherset','origin'],axis = 1).values
    y = df['ground.truth'].values
    y = [0 if i == 'nominal' else 1 for i in y]
    
    outliers_fraction = min(np.count_nonzero(y) / len(y),0.5)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [file[-8:-4], x.shape[0], x.shape[1], outliers_percentage]
    prn_list = [file[-8:-4], x.shape[0], x.shape[1], outliers_percentage]
    time_list = [file[-8:-4], x.shape[0], x.shape[1], outliers_percentage]
    
    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
        contamination=outliers_fraction),
        'Cluster-based Local Outlier Factor': CBLOF(
            contamination=outliers_fraction, check_estimator=False,
            random_state=random_state),
        'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                          random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'Minimum Covariance Determinant (MCD)': MCD(
            contamination=outliers_fraction, random_state=random_state),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
        'Principal Component Analysis (PCA)': PCA(
            contamination=outliers_fraction, random_state=random_state),
    }

    for clf_name, clf in classifiers.items():
        try:
            t0 = time()
            clf.fit(X_train_norm)
            test_scores = clf.decision_function(X_test_norm)
            t1 = time()
            duration = round(t1 - t0, ndigits=4)
            roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
            prn = round(precision_n_scores(y_test, test_scores), ndigits=4)
        except Exception as e:
            roc = 0
            prn = 0
            duration = 0

        print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
              'execution time: {duration}s'.format(
            clf_name=clf_name, roc=roc, prn=prn, duration=duration))
        time_list.append(duration)
        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)

将结果导出保存为csv文件。

In [ ]:
roc_df.to_csv('roc_df.csv', index=False)
prn_df.to_csv('prn_df.csv', index=False)
time_df.to_csv('time_df.csv', index=False)

## 结果及分析

### Abalone benchmarks

In [16]:
abalone_roc_df = pd.read_csv('data/abalone_roc_df.csv',index_col = 'Data')
abalone_prn_df = pd.read_csv('data/abalone_prn_df.csv',index_col = 'Data')
abalone_time_df = pd.read_csv('data/abalone_time_df.csv',index_col = 'Data')

#### Precision_N_Scores

In [17]:
abalone_prn_df.head()

,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
Data,,,,,,,,,,,,,
1,1888,9,48.3581,0.6438,0.5620,0.5462,0.4776,0.4222,0.6755,0.5462,0.7124,0.4960,0.4697
2,1888,9,50.0000,0.6640,0.5079,0.5265,0.4921,0.4101,0.6587,0.5317,0.6772,0.4550,0.4603
3,1888,9,50.0000,0.6812,0.5784,0.6041,0.4072,0.4473,0.6864,0.6093,0.7044,0.4884,0.4447
4,1888,9,47.1928,0.6788,0.5335,0.5447,0.4721,0.4413,0.6732,0.5028,0.7123,0.4385,0.4358
5,1888,9,49.3644,0.6096,0.5169,0.5365,0.3848,0.4242,0.6404,0.5253,0.6601,0.4326,0.4129


模型的平均准确率：

In [18]:
res = abalone_prn_df.loc[:,"ABOD":"PCA"].apply(lambda x: x.mean())
res.sort_values(ascending=False)

KNN        0.417162
MCD        0.374936
OCSVM      0.372675
FB         0.362493
CBLOF      0.359890
LOF        0.354568
ABOD       0.336974
PCA        0.314957
HBOS       0.309340
IForest    0.292431
dtype: float64

由以上结果可知，KNN的准确率最高，平均为0.417,其他的模型准确率结果在0.3-0.37之间

#### ROC_AUC_Score

In [19]:
abalone_roc_df.head()

,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
Data,,,,,,,,,,,,,
1,1888,9,48.3581,0.7144,0.5919,0.5696,0.4839,0.4096,0.7449,0.5617,0.7942,0.4871,0.4556
2,1888,9,50.0000,0.7427,0.5308,0.5321,0.4566,0.3959,0.7416,0.5299,0.7536,0.4536,0.4412
3,1888,9,50.0000,0.7414,0.6171,0.6115,0.3560,0.3996,0.7673,0.6114,0.7724,0.4896,0.4259
4,1888,9,47.1928,0.7562,0.5817,0.5744,0.5294,0.4602,0.7752,0.5429,0.8030,0.4766,0.4523
5,1888,9,49.3644,0.7097,0.5776,0.5802,0.3720,0.4365,0.7364,0.5785,0.7557,0.4628,0.4306


模型的平均ROC/AUC值：

In [20]:
res = abalone_roc_df.loc[:,"ABOD":"PCA"].apply(lambda x: x.mean())
res.sort_values(ascending=False)

MCD        0.825583
KNN        0.814653
ABOD       0.786419
FB         0.784686
LOF        0.773893
OCSVM      0.772424
HBOS       0.753675
CBLOF      0.741119
PCA        0.723892
IForest    0.720694
dtype: float64

由以上结果可知，MCD的ROC值最高,为0.825, 紧跟其后的是KNN,结果为0.814。其余模型的结果都在0.7-0.8之间。

#### 耗费时间

In [21]:
abalone_time_df.head()

,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
Data,,,,,,,,,,,,,
1,1888,9,48.3581,1.3896,1.3649,0.2370,0.9226,0.3102,0.0537,0.0218,0.9981,0.1099,0.0044
2,1888,9,50.0000,0.3568,0.1187,0.2375,0.0034,0.2763,0.0568,0.0243,0.8782,0.0830,0.0030
3,1888,9,50.0000,0.3394,0.1333,0.1918,0.0037,0.2854,0.0567,0.0235,0.9234,0.1017,0.0043
4,1888,9,47.1928,0.3189,0.1059,0.1855,0.0034,0.2805,0.0632,0.0284,0.9739,0.0855,0.0035
5,1888,9,49.3644,0.3918,0.0966,0.2057,0.0034,0.2744,0.0567,0.0230,0.8784,0.0837,0.0026


平均耗费时间：

In [22]:
res = abalone_time_df.loc[:,"ABOD":"PCA"].apply(lambda x: x.mean())
res.sort_values(ascending=False)

MCD        0.963553
FB         0.450495
ABOD       0.324901
IForest    0.257274
CBLOF      0.112568
OCSVM      0.089059
KNN        0.084967
LOF        0.056097
HBOS       0.005670
PCA        0.003200
dtype: float64

由以上结果可知，MCD花费的时间最长，为0.963，其花费的时间最长，而对应的ROC也是最高的。而PCA所耗费的时间最少，KNN所耗费时间也较低，为0.089。  
由此可见，KNN的效率相对来说非常高，在较短的时间内，能获得较高的精确度。而MCD做为ROC值最高的模型，相对其耗费时间也是最高的。

### Fault benchmarks